In [23]:
import numpy as np
import json
import tensorflow.keras as K
import tensorflow as tf
import cv2
from PIL import Image
import pandas as pd
import json
from sklearn.utils import shuffle


In [37]:
df = pd.read_csv('../malaria_data/RBC20k_no_diff_leuk.csv')
df

,Unnamed: 0,path,min_r,min_c,max_r,max_c,r_len,c_len,category,box_area,c_by_r,r_by_c
0,54,/images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,52,279,200,409,148,130,trophozoite,19240,0.878378,1.138462
1,61,/images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,441,1037,578,1155,137,118,trophozoite,16166,0.861314,1.161017
2,152,/images/6b14c855-8561-417c-97a4-63fa552842fd.png,76,482,219,626,143,144,schizont,20592,1.006993,0.993056
3,156,/images/13099edb-35d9-438f-b093-2cf2ebf9d255.png,208,324,337,446,129,122,ring,15738,0.945736,1.057377
4,235,/images/2559636b-f01a-4414-93da-210c3b12d153.png,130,866,292,1014,162,148,schizont,23976,0.913580,1.094595
...,...,...,...,...,...,...,...,...,...,...,...,...
21505,70595,/images/4ce5a070-9ede-4bb1-9ca2-1bae5746776b.png,631,1218,734,1321,103,103,red blood cell,10609,1.000000,1.000000
21506,57348,/images/a88a48b6-f85c-45c1-b42f-79716c8463f1.png,286,1032,377,1127,91,95,red blood cell,8645,1.043956,0.957895
21507,78710,/images/76a8a5f5-1328-4539-a131-baf877f40cf3.png,169,659,280,760,111,101,red blood cell,11211,0.909910,1.099010
21508,56302,/images/6b473806-e936-4b19-98cf-a2b94cac35ff.png,644,1077,756,1188,112,111,red blood cell,12432,0.991071,1.009009


In [38]:
df.category.value_counts()

red blood cell    20000
trophozoite         993
ring                305
gametocyte          107
schizont            105
Name: category, dtype: int64

In [39]:
df['category'] = df['category'].replace({
    'red blood cell': 'uninfected',
    'trophozoite': 'infected',
    'ring': 'infected',
    'gametocyte': 'infected',
    'schizont': 'infected'
})

In [40]:
df.category.value_counts()

uninfected    20000
infected       1510
Name: category, dtype: int64

In [34]:
df = df[df['category'] != 'leukocyte']

In [35]:
df

,Unnamed: 0,path,min_r,min_c,max_r,max_c,r_len,c_len,category,box_area,c_by_r,r_by_c
0,54,/images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,52,279,200,409,148,130,infected,19240,0.878378,1.138462
1,61,/images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,441,1037,578,1155,137,118,infected,16166,0.861314,1.161017
2,152,/images/6b14c855-8561-417c-97a4-63fa552842fd.png,76,482,219,626,143,144,infected,20592,1.006993,0.993056
3,156,/images/13099edb-35d9-438f-b093-2cf2ebf9d255.png,208,324,337,446,129,122,infected,15738,0.945736,1.057377
4,235,/images/2559636b-f01a-4414-93da-210c3b12d153.png,130,866,292,1014,162,148,infected,23976,0.913580,1.094595
...,...,...,...,...,...,...,...,...,...,...,...,...
21582,70595,/images/4ce5a070-9ede-4bb1-9ca2-1bae5746776b.png,631,1218,734,1321,103,103,uninfected,10609,1.000000,1.000000
21583,57348,/images/a88a48b6-f85c-45c1-b42f-79716c8463f1.png,286,1032,377,1127,91,95,uninfected,8645,1.043956,0.957895
21584,78710,/images/76a8a5f5-1328-4539-a131-baf877f40cf3.png,169,659,280,760,111,101,uninfected,11211,0.909910,1.099010
21585,56302,/images/6b473806-e936-4b19-98cf-a2b94cac35ff.png,644,1077,756,1188,112,111,uninfected,12432,0.991071,1.009009


In [36]:
df.category.value_counts()

uninfected    20000
infected       1510
Name: category, dtype: int64

                                                   path  min_r  min_c  max_r  \
0      /images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png   1057   1440   1158   
1      /images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png    868   1303    971   
2      /images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png    578    900    689   
3      /images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png    304    611    408   
4      /images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png    198    881    312   
...                                                 ...    ...    ...    ...   
80108  /images/99e979d4-ee97-4d7f-9da6-00bead40db4d.png    757    520    873   
80109  /images/99e979d4-ee97-4d7f-9da6-00bead40db4d.png    308    808    406   
80110  /images/99e979d4-ee97-4d7f-9da6-00bead40db4d.png    510    453    595   
80111  /images/99e979d4-ee97-4d7f-9da6-00bead40db4d.png      0   1260     87   
80112  /images/99e979d4-ee97-4d7f-9da6-00bead40db4d.png    740      0    852   

       max_c  r_len  c_len        categ